In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Desktop\\Kidney_disease-DL-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Desktop\\Kidney_disease-DL-'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/latab2130/Kidney_disease-DL-.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="latab2130"
os.environ["MLFLOW_TRACKING_PASSWORD"]="91b44ad7ed224c08b4e612ca139c7deb91ce03dc"


In [6]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kideny_data",
            mlflow_uri="https://dagshub.com/latab2130/Kidney_disease-DL-.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config):
        """
        Initializes the Evaluation class with the provided configuration.

        Args:
            config (EvaluationConfig): Configuration object containing all necessary parameters.
        """
        self.config = config

    def _valid_generator(self):
        """
        Prepares the validation data generator with the specified parameters.
        """
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        """
        Loads a saved model from the specified path.

        Args:
            path (Path): Path to the saved model.

        Returns:
            tf.keras.Model: Loaded TensorFlow model.
        """
        return tf.keras.models.load_model(path)

    def evaluation(self):
        """
        Evaluates the model on the validation data and saves the evaluation score.
        """
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        """
        Saves the evaluation scores (loss and accuracy) to a JSON file.
        """
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        """
        Logs parameters, metrics, and the model into MLflow for tracking and registry.
        """
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Log parameters
            mlflow.log_params(self.config.all_params)

            # Log metrics
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            # Log model
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-01-23 15:18:51,569: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-23 15:18:51,579: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-23 15:18:51,582: INFO: common: created directory at: artifacts]
Found 2207 images belonging to 2 classes.
 64/138 [============>.................] - ETA: 3:57 - loss: 59.3657 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [10]:
import mlflow

# Set tracking URI
mlflow.set_tracking_uri("https://dagshub.com/latab2130/Kidney_disease-DL-.mlflow")

# Test experiment creation
mlflow.set_experiment("trail_experiment")
with mlflow.start_run():
    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 0.85)

print("Connection successful!")


Connection successful!


In [7]:
import os
print(os.environ.get("MLFLOW_TRACKING_USERNAME"))
print(os.environ.get("MLFLOW_TRACKING_PASSWORD"))


latab2130
91b44ad7ed224c08b4e612ca139c7deb91ce03dc
